In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
matches = pd.read_csv('all_t20i_05-16.csv')
match_date = matches.set_index('match_id').date.to_dict()

In [36]:
files = sorted(match_date.items(), key=lambda value: value[1])

In [41]:
import os.path

In [48]:
all_data = pd.DataFrame()
for idx, fname in enumerate(files):
    if os.path.exists('cleaned_data/%d_summary.csv'%fname[0]):
        ind_game = pd.read_csv('cleaned_data/%d_summary.csv'%fname[0])#.sort_values('inning').sort_values('ovr')
        ind_game['date'] = fname[1]
        all_data = all_data.append(ind_game)
    if idx%50 == 0:
        print idx, len(files)

0 497
50 497
100 497
150 497
200 497
250 497
300 497
350 497
400 497
450 497


In [49]:
all_data = all_data.reset_index(drop=True)
cols = all_data.columns.tolist()
all_data = all_data[cols[1:]]

In [51]:
all_data['year'] = all_data['date'].apply(lambda x: x.split('-')[0])

In [234]:
batsman_team_dict = all_data.drop_duplicates(subset = ['batsman_name']).set_index('batsman_name')['batting_team'].to_dict()
bowler_team_dict = all_data.drop_duplicates(subset = ['bowler_name']).set_index('bowler_name')['bowling_team'].to_dict()

In [261]:
major_teams = ['Australia', 'New Zealand', 'England', 'South Africa',
       'West Indies', 'Sri Lanka', 'Pakistan', 'India', 'Bangladesh']

In [267]:
batsman_innings = pd.DataFrame()
batsman_innings['runs_scored'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].agg('sum')
batsman_innings['num_out'] = all_data.groupby(['batsman_name', 'date'])['wicket'].agg('sum')
batsman_innings['balls_faced'] = all_data.groupby(['batsman_name', 'date'])['ovr'].count()
batsman_innings['num_fours'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 4 for i in x.tolist()]))
batsman_innings['num_sixes'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 6 for i in x.tolist()]))
batsman_innings['num_dots'] = all_data.groupby(['batsman_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
batsman_innings = batsman_innings.reset_index()
batsman_innings['year'] = batsman_innings['date'].apply(lambda x: int(x.split('-')[0]))
batsman_innings['year'] = batsman_innings['year'].apply(lambda x: '05-07' if x <= 2007 else '08-10' if x <= 2010 else '11-13' if x <=2013 else '14-16')
batsman_innings['strike_rate'] = 100.*batsman_innings.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)

In [268]:
batsman_params = ['runs_scored', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'num_out']
batsman_year = batsman_innings.groupby(['batsman_name', 'year'])[batsman_params].agg('sum')\
    .join(batsman_innings.groupby(['batsman_name', 'year'])['date'].nunique())
batsman_year.columns = batsman_year.columns.tolist()[:-1] + ['num_innings']
batsman_year = batsman_year.reset_index()
batsman_year['team'] = batsman_year['batsman_name'].apply(batsman_team_dict.get)

### Impact Definition

Define impact better --> Quantify in terms of chances of winning

Different stages of the game (bowler and batsmen)

In [271]:
batsman_year['average'] = batsman_year.apply(lambda row: row['runs_scored']/max(row['num_out'], 1), axis=1)
batsman_year['runs_per_inning'] = batsman_year['runs_scored']/batsman_year['num_innings']
batsman_year['strike_rate'] = 100.*batsman_year.apply(lambda row: row['runs_scored']/max(row['balls_faced'], 1), axis=1)
batsman_year['impact'] = batsman_year['strike_rate']*batsman_year['runs_per_inning']
batsman_year['prop_dot'] = 100.*batsman_year['num_dots']/batsman_year['balls_faced']
batsman_year['balls_per_six'] = batsman_year['balls_faced']/batsman_year['num_sixes']
batsman_year['balls_per_boundary'] = batsman_year['balls_faced']/(batsman_year['num_sixes'] + batsman_year['num_fours'])
batsman_year['prop_run_boundary'] = 100.*(6.*batsman_year['num_sixes'] + 4.*batsman_year['num_fours'])/batsman_year['runs_scored']
batsman_year['prop_run_six'] = 100.*(6.*batsman_year['num_sixes'])/batsman_year['runs_scored']

In [272]:
batsman_year[batsman_year.batsman_name.str.contains('Rohit')]

,batsman_name,year,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,num_innings,team,average,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,balls_per_boundary,prop_run_boundary,prop_run_six
1118,Rohit Sharma,05-07,88,62,9,4,25,0,3,India,88.000000,29.333333,141.935484,4163.440860,40.322581,15.500000,4.769231,68.181818,27.272727
1119,Rohit Sharma,08-10,247,213,19,10,93,9,12,India,27.444444,20.583333,115.962441,2386.893584,43.661972,21.300000,7.344828,55.060729,24.291498
1120,Rohit Sharma,11-13,204,160,14,7,48,7,13,India,29.142857,15.692308,127.500000,2000.769231,30.000000,22.857143,7.619048,48.039216,20.588235
1121,Rohit Sharma,14-16,527,419,50,17,158,12,14,India,43.916667,37.642857,125.775656,4734.555063,37.708831,24.647059,6.253731,57.305503,19.354839


In [273]:
batsman_year[batsman_year.batsman_name.str.contains('Dhoni')]

,batsman_name,year,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,num_innings,team,average,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,balls_per_boundary,prop_run_boundary,prop_run_six
747,MS Dhoni,05-07,163,129,13,4,37,3,8,India,54.333333,20.375000,126.356589,2574.515504,28.682171,32.250000,7.588235,46.625767,14.723926
748,MS Dhoni,08-10,278,272,13,9,105,9,16,India,30.888889,17.375000,102.205882,1775.827206,38.602941,30.222222,12.363636,38.129496,19.424460
749,MS Dhoni,11-13,331,290,27,7,114,8,16,India,41.375000,20.687500,114.137931,2361.228448,39.310345,41.428571,8.529412,45.317221,12.688822
750,MS Dhoni,14-16,138,106,10,6,39,4,11,India,34.500000,12.545455,130.188679,1633.276158,36.792453,17.666667,6.625000,55.072464,26.086957


In [274]:
batsman_year[batsman_year.batsman_name.str.contains('Virat')]

,batsman_name,year,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,num_innings,team,average,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,balls_per_boundary,prop_run_boundary,prop_run_six
1407,Virat Kohli,08-10,26,21,3,1,9,0,1,India,26.0000,26.000000,123.809524,3219.047619,42.857143,21.000000,5.250000,69.230769,23.076923
1408,Virat Kohli,11-13,561,437,69,8,146,16,18,India,35.0625,31.166667,128.375286,4001.029748,33.409611,54.625000,5.675325,57.754011,8.556150
1409,Virat Kohli,14-16,628,458,52,18,122,8,12,India,78.5000,52.333333,137.117904,7175.836972,26.637555,25.444444,6.542857,50.318471,17.197452


In [275]:
batsman_year[batsman_year.batsman_name.str.contains('Yuvraj')]

,batsman_name,year,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,num_innings,team,average,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,balls_per_boundary,prop_run_boundary,prop_run_six
1446,Yuvraj Singh,05-07,179,103,10,15,36,4,6,India,44.750000,29.833333,173.786408,5184.627832,34.951456,6.866667,4.120000,72.625698,50.279330
1447,Yuvraj Singh,08-10,376,278,23,23,110,14,15,India,26.857143,25.066667,135.251799,3390.311751,39.568345,12.086957,6.043478,61.170213,36.702128
1448,Yuvraj Singh,11-13,313,219,20,21,83,9,11,India,34.777778,28.454545,142.922374,4066.791200,37.899543,10.428571,5.341463,65.814696,40.255591
1449,Yuvraj Singh,14-16,125,132,9,6,65,7,8,India,17.857143,15.625000,94.696970,1479.640152,49.242424,22.000000,8.800000,57.600000,28.800000


In [277]:
params = ['batsman_name', 'team', 'year', 'runs_scored', 'balls_faced', 'num_innings', 'runs_per_inning',
      'strike_rate', 'impact', 'prop_dot', 'balls_per_six', 'prop_run_six']
batsman_year[(batsman_year.balls_faced >= 50) & (batsman_year.num_innings >= 5) & (batsman_year.team.isin(major_teams))]\
    .sort_values('impact', ascending=False)[params]\
    .round({'runs_per_inning': 1, 'strike_rate': 1, 'impact': 0,
            'prop_dot': 1, 'balls_per_six': 1, 'prop_run_six': 1}).head(15)

,batsman_name,team,year,runs_scored,balls_faced,num_innings,runs_per_inning,strike_rate,impact,prop_dot,balls_per_six,prop_run_six
1409,Virat Kohli,India,14-16,628,458,12,52.3,137.1,7176,26.6,25.4,17.2
6,Aaron Finch,Australia,11-13,366,211,9,40.7,173.5,7054,30.8,12.4,27.9
243,Chris Gayle,West Indies,05-07,193,126,5,38.6,153.2,5913,42.9,11.5,34.2
196,Brendon McCullum,New Zealand,11-13,781,536,20,39.0,145.7,5690,36.9,14.9,27.7
108,Andrew Symonds,Australia,05-07,337,203,10,33.7,166.0,5595,25.1,20.3,17.8
1172,Sanath Jayasuriya,Sri Lanka,05-07,246,155,7,35.1,158.7,5578,41.9,15.5,24.4
246,Chris Gayle,West Indies,14-16,407,305,10,40.7,133.4,5431,47.2,11.3,39.8
684,Kevin Pietersen,England,08-10,627,463,16,39.2,135.4,5307,33.3,27.2,16.3
1446,Yuvraj Singh,India,05-07,179,103,6,29.8,173.8,5185,35.0,6.9,50.3
1220,Shane Watson,Australia,11-13,651,435,19,34.3,149.7,5128,41.8,10.1,39.6


In [280]:
bowler_spells = pd.DataFrame()
bowler_spells['runs_conceded'] = all_data.groupby(['bowler_name', 'date'])['runs_batter'].agg('sum')
bowler_spells['wickets_taken'] = all_data.groupby(['bowler_name', 'date'])['wicket'].agg('sum')
bowler_spells['balls_bowled'] = all_data.groupby(['bowler_name', 'date'])['ovr'].count()
bowler_spells['num_dots'] = all_data.groupby(['bowler_name', 'date'])['runs_batter'].apply(lambda x: sum([i == 0 for i in x.tolist()]))
bowler_spells = bowler_spells.reset_index()
bowler_spells['year'] = bowler_spells['date'].apply(lambda x: int(x.split('-')[0]))
bowler_spells['year'] = bowler_spells['year'].apply(lambda x: '05-07' if x <= 2007 else '08-10' if x <= 2010 else '11-13' if x <=2013 else '14-16')
bowler_spells['economy_rate'] = 6.*bowler_spells.apply(lambda row: row['runs_conceded']/max(row['balls_bowled'], 1), axis=1)

In [281]:
bowler_params = ['runs_conceded', 'balls_bowled', 'wickets_taken', 'num_dots']
bowler_year = bowler_spells.groupby(['bowler_name', 'year'])[bowler_params].agg('sum')\
    .join(bowler_spells.groupby(['bowler_name', 'year'])['date'].nunique())
bowler_year.columns = bowler_year.columns.tolist()[:-1] + ['num_spells']
bowler_year = bowler_year.reset_index()
bowler_year['team'] = bowler_year['bowler_name'].apply(bowler_team_dict.get)

In [282]:
bowler_year['average'] = bowler_year['runs_conceded']/bowler_year['wickets_taken']
bowler_year['wickets_per_spell'] = bowler_year['wickets_taken']/bowler_year['num_spells']
bowler_year['strike_rate'] = bowler_year['balls_bowled']/bowler_year['wickets_taken']
bowler_year['economy_rate'] = 6.*bowler_year['runs_conceded']/bowler_year['balls_bowled']
bowler_year['prop_dot'] = 100.*bowler_year['num_dots']/bowler_year['balls_bowled']

In [283]:
bowler_year[bowler_year.bowler_name.str.contains('Ashwin')]

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
751,Ravichandran Ashwin,08-10,68,50,2,16,2,India,34.000000,1.000000,25.000000,8.160000,32.000000
752,Ravichandran Ashwin,11-13,483,405,13,162,17,India,37.153846,0.764706,31.153846,7.155556,40.000000
753,Ravichandran Ashwin,14-16,325,359,31,178,15,India,10.483871,2.066667,11.580645,5.431755,49.582173


In [293]:
bowler_year[bowler_year.bowler_name.str.contains('Harbhajan')]

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
334,Harbhajan Singh,05-07,212,185,12,60,8,India,17.666667,1.500000,15.416667,6.875676,32.432432
335,Harbhajan Singh,08-10,284,303,7,125,13,India,40.571429,0.538462,43.285714,5.623762,41.254125
336,Harbhajan Singh,11-13,55,62,6,33,3,India,9.166667,2.000000,10.333333,5.322581,53.225806
337,Harbhajan Singh,14-16,40,51,2,26,2,India,20.000000,1.000000,25.500000,4.705882,50.980392


In [291]:
bowler_year[bowler_year.bowler_name.str.contains('Umar Gul')]

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
1003,Umar Gul,05-07,153,193,12,94,8,Pakistan,12.750000,1.500000,16.083333,4.756477,48.704663
1004,Umar Gul,08-10,556,545,38,258,26,Pakistan,14.631579,1.461538,14.342105,6.121101,47.339450
1005,Umar Gul,11-13,438,371,30,159,18,Pakistan,14.600000,1.666667,12.366667,7.083558,42.857143
1006,Umar Gul,14-16,221,158,10,67,8,Pakistan,22.100000,1.250000,15.800000,8.392405,42.405063


In [288]:
bowler_year[(bowler_year.balls_bowled >= 100) & (bowler_year.team.isin(major_teams))].sort_values('economy_rate')

,bowler_name,year,runs_conceded,balls_bowled,wickets_taken,num_dots,num_spells,team,average,wickets_per_spell,strike_rate,economy_rate,prop_dot
199,Dale Steyn,11-13,179,234,17,139,11,South Africa,10.529412,1.545455,13.764706,4.589744,59.401709
1003,Umar Gul,05-07,153,193,12,94,8,Pakistan,12.750000,1.500000,16.083333,4.756477,48.704663
362,Imran Tahir,11-13,104,131,7,47,6,South Africa,14.857143,1.166667,18.714286,4.763359,35.877863
647,Mustafizur Rahman,14-16,140,172,10,97,7,Bangladesh,14.000000,1.428571,17.200000,4.883721,56.395349
824,Samuel Badree,14-16,233,284,20,156,12,West Indies,11.650000,1.666667,14.200000,4.922535,54.929577
206,Daniel Vettori,08-10,391,469,22,208,20,New Zealand,17.772727,1.100000,21.318182,5.002132,44.349680
935,Sunil Narine,14-16,296,350,17,168,15,West Indies,17.411765,1.133333,20.588235,5.074286,48.000000
808,Sachithra Senanayake,11-13,150,172,14,77,8,Sri Lanka,10.714286,1.750000,12.285714,5.232558,44.767442
874,Shaun Tait,08-10,353,396,27,229,16,Australia,13.074074,1.687500,14.666667,5.348485,57.828283
634,Morne Morkel,05-07,114,126,10,65,5,South Africa,11.400000,2.000000,12.600000,5.428571,51.587302


In [310]:
bowling_overall = bowler_year[bowler_year.team.isin(major_teams)].groupby('year')[['runs_conceded', 'balls_bowled', 'wickets_taken']].agg('sum')

In [311]:
bowling_overall['economy_rate'] = 6.*bowling_overall['runs_conceded']/bowling_overall['balls_bowled']
bowling_overall['stike_rate'] = bowling_overall['balls_bowled']/bowling_overall['wickets_taken']
bowling_overall['average'] = bowling_overall['runs_conceded']/bowling_overall['wickets_taken']

In [312]:
bowling_overall

,runs_conceded,balls_bowled,wickets_taken,economy_rate,stike_rate,average
year,,,,,,
05-07,12368,10497,576,7.069448,18.223958,21.472222
08-10,27222,24016,1326,6.800966,18.111614,20.529412
11-13,29385,24940,1298,7.069366,19.214176,22.638675
14-16,23805,19876,1058,7.186054,18.786389,22.500000


In [316]:
batting_overall = batsman_year[batsman_year.team.isin(major_teams)].groupby('year')[['runs_scored', 'balls_faced', 'num_fours', 'num_sixes', 'num_dots', 'num_out']].agg('sum')

In [317]:
batting_overall['strike_rate'] = 100.*batting_overall['runs_scored']/batting_overall['balls_faced']
batting_overall['average'] = 100.*batting_overall['runs_scored']/batting_overall['num_out']

In [319]:
batting_overall

,runs_scored,balls_faced,num_fours,num_sixes,num_dots,num_out,strike_rate,average
year,,,,,,,,
05-07,12653,10399,1166,446,4401,555,121.675161,2279.819820
08-10,28238,24354,2415,933,10241,1290,115.948099,2188.992248
11-13,30111,25155,2650,1019,10247,1235,119.701849,2438.137652
14-16,24054,19811,2131,854,8016,1042,121.417394,2308.445298
